# Imports

In [1]:
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import accuracy_score

2024-06-02 13:45:42.967860: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-02 13:45:43.130950: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-02 13:45:43.719876: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 13:45:44.965579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## set data directories

In [2]:
train_dir = './data/train'
test_dir = './data/test'

## Images's processing params

In [3]:
img_height, img_width = 150, 150
batch_size = 32

## Using ImageDataGenerator to load and preprocess images

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

# Fonction to load and preprocess images

In [5]:
def load_and_preprocess_images(directory):
    generator = datagen.flow_from_directory(
        directory,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    images, labels = [], []
    
    # Obtenir un itérateur sur le générateur
    generator_iter = iter(generator)
    
    # Utiliser next() sur l'itérateur
    for _ in range(len(generator)):
        batch = next(generator_iter)
        images.append(batch[0])
        labels.append(batch[1])
    
    images = np.concatenate(images)
    labels = np.concatenate(labels)
    return images, labels

## load train and test images

In [6]:
train_images, train_labels = load_and_preprocess_images(train_dir)
test_images, test_labels = load_and_preprocess_images(test_dir)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Resizing images to transform tehem to feature vectors

In [7]:
train_features = train_images.reshape(train_images.shape[0], -1)
test_features = test_images.reshape(test_images.shape[0], -1)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Set the hyperparameters to test

In [12]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}


# Initialize the logistic regression model

In [13]:
log_reg = LogisticRegression(max_iter=1000)

## Configure GridSearchCV with cross validation

In [14]:
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy')

## Run GridSearchCV to find the best hyperparameters

In [25]:
grid_search.fit(train_features, train_labels)

: 